In [1]:
import numpy as np
import pandas as pd
import joblib  # per caricare i modelli Random Forest salvati con dump(...)

from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem import rdFingerprintGenerator

######################################
# 1) Funzione smiles_to_fp
######################################
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.MolStandardize import rdMolStandardize


from joblib import dump

from rdkit import RDLogger

# Imposta il livello di log a ERROR (ignora INFO e WARNING)
lg = RDLogger.logger()
lg.setLevel(RDLogger.ERROR)


def standardize_mol(mol):
    """
    Esempio minimal di standardizzazione:
    1) Cleanup chimico di base
    2) Tieni solo il frammento più grande
    (Volendo puoi aggiungere tautomer enumeration, neutralizzazione, etc.)
    """
    # 1) Pulizia di base
    mol = rdMolStandardize.Cleanup(mol)
    # 2) Tieni solo il frammento principale
    lfc = rdMolStandardize.LargestFragmentChooser()
    mol = lfc.choose(mol)
    return mol

def smiles_to_fp(smiles, 
                 nBits=1024, 
                 radius=2, 
                 use_MACCS=False, 
                 standardize=True):
    """
    Converte uno SMILES in fingerprint Morgan (di default).
    Se use_MACCS=True, concatena anche il fingerprint MACCS
    Se standardize=True, prova a "ripulire" la molecola prima di calcolare il FP
    """

    # Converte da SMILES a Mol
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        # Se la conversione fallisce, restituisci un vettore di zeri
        # con dimensione pari a nBits (Morgan) o nBits + 166 se concateni MACCS
        maccs_size = 166 if use_MACCS else 0
        return np.zeros(nBits + maccs_size, dtype=np.uint8)

    # Standardizzazione (opzionale)
    if standardize:
        mol = standardize_mol(mol)

    # Genera fingerprint Morgan (usando rdFingerprintGenerator o AllChem)
    fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
    fp_morgan = fpgen.GetFingerprint(mol)
    arr_morgan = np.zeros((nBits,), dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(fp_morgan, arr_morgan)

    if not use_MACCS:
        # Ritorna solo Morgan
        return arr_morgan

    # Altrimenti, calcola MACCS e concatena
    maccs_fp = MACCSkeys.GenMACCSKeys(mol)
    arr_maccs = np.zeros((maccs_fp.GetNumBits(),), dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(maccs_fp, arr_maccs)

    # Concatenazione Morgan + MACCS
    combined_fp = np.concatenate([arr_morgan, arr_maccs])
    return combined_fp



df_test = pd.read_csv("../data/smiles_test.csv")  # con colonna "smiles"

# 3) Costruisci fingerprint per tutti
X_test = np.array([smiles_to_fp(s, 
                           nBits=1024, 
                           radius=2, 
                           use_MACCS=True, 
                           standardize=True) 
              for s in df_test["smiles"]],
             dtype=np.float32)
nBits = 1024  # deve corrispondere a quello usato in training

######################################
# 3) Carica i modelli e predici
######################################
task_probs = []  # per salvare le predizioni di ogni task

for i in range(1, 12):
    model_path = f"best_models/rf_task{i}.joblib"
    try:
        rf_model = joblib.load(model_path)
    except FileNotFoundError:
        print(f"Modello {model_path} non trovato. Inserisco colonna di NaN.")
        # Se il modello non esiste, puoi mettere 0.5 oppure np.nan
        probs_i = np.full(len(X_test), np.nan, dtype=np.float32)
        task_probs.append(probs_i)
        continue
    
    # Previsione di probabilità (classe=1) su X_test
    y_proba = rf_model.predict_proba(X_test)[:, 1]  # shape (N,)
    task_probs.append(y_proba)

######################################
# 4) Costruiamo il DataFrame di output
######################################
# Trasponiamo per avere shape (N, 11)
probs_matrix = np.column_stack(task_probs)  # (N, 11)
task_names = [f"task{i}" for i in range(1, 12)]
df_preds = pd.DataFrame(probs_matrix, columns=task_names)

######################################
# 5) Inserisci colonna di indici (senza nome)
######################################
df_preds.insert(0, '', range(len(df_preds)))  # colonna di indici 0..N-1

######################################
# 6) Salva in CSV
######################################
df_preds.to_csv("predictions_rf.csv", index=False)
print("Predizioni salvate in predictions_rf.csv")


Predizioni salvate in predictions_rf.csv
